# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [19]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [20]:
LFP_SPECTRAL_DF.columns

Index(['video_name', 'current_subject', '10s_after_tone_frame',
       '10s_before_tone_frame', 'agent_locations', 'agent_nose',
       'agent_nose_to_reward_port', 'agent_tail_base', 'agent_thorax',
       'agent_thorax_velocity',
       ...
       'vHPC_gamma_phase', 'vHPC_lfp_trace',
       'vHPC_mPFC_granger_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'vHPC_power_gamma',
       'vHPC_power_theta', 'vHPC_theta_band', 'vHPC_theta_phase',
       'video_timestamps', 'cluster_timestamp'],
      dtype='object', length=220)

In [21]:
TRIAL_LABELS_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [22]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,rewarded,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [23]:
LFP_SPECTRAL_DF.head()

,video_name,current_subject,10s_after_tone_frame,10s_before_tone_frame,agent_locations,agent_nose,agent_nose_to_reward_port,agent_tail_base,agent_thorax,agent_thorax_velocity,...,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.1,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[54.047201234350496, 32.48462136905939], [52...","[[57.31849083590113, 33.645587310504695], [56....","[4.960474, 5.34718, 5.802983, 6.407976, 7.1417...","[[46.630318040576064, 30.523901100123272], [46...","[[48.61806735273943, 32.66608744832453], [48.1...","[7.0735817, 9.794592, 12.977724, 16.4886, 20.1...",...,"[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
1,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.3,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[62.81573570925102, 33.49031610207749], [59....","[[60.5507800219845, 35.26465151447588], [60.78...","[0.53517926, 0.50113046, 0.48567954, 0.4737239...","[[66.06126838242345, 28.880162680346388], [66....","[[64.05699608055423, 31.17933144091776], [64.0...","[0.07300971, 0.08296164, 0.08879429, 0.0909930...",...,"[1.1501461, 1.6517024, 1.9768442, 2.2265384, 2...","[-100.619995, -102.96, -129.87, -165.75, -191....","[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[34077145, 34077145, 34078531, 34079917, 34079..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[56.40002042465876, 33.47263135770957], [55....","[[60.438020774390075, 35.57292836881663], [60....","[1.4890755, 1.1949695, 0.93336076, 0.738537, 0...","[[48.80137642581422, 31.16922356981376], [49.0...","[[51.35546426976598, 33.93773101458849], [52.3...","[10.536535, 10.738277, 10.965685, 11.176429, 1...",...,"[0.023950227, 0.96471804, 1.2889154, 1.6205887...","[-74.685, -113.49, -156.78, -194.21999, -266.1...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.0013803464, 0.0079310145, 0.009366615, 0.0...","[0.00366902, nan, nan, 0.002184509, 0.00216185...","[0.046842232, nan, nan, 0.023685867, 0.0444350...","[-0.37066078, -0.35752764, -0.3432352, -0.3277...","[2.7155704, -2.8761566, -2.806554, -2.548516, ...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[61.39644296346538, 31.82081289424359], [58....","[[62.13302320168461, 34.13495024512682], [61.3...","[2.1410906, 2.5980964, 3.2073143, 3.9916637, 4...","[[50.83958858240558, 27.427984293098355], [50....","[[54.225662928143386, 30.269937128249303], [54...","[5.105928, 5.1050973, 5.082959, 5.072279, 5.09...",...,"[2.0565047, 2.3807256, 2.6063833, 2.87997, 3.1...","[53.234997, 80.145, 106.079994, 139.42499, 206...","[[2.7552054, 1.5484817, 2.318468, 0.67259854, ...

# Dropping redundant columns

In [24]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [25]:
dropping_columns

['session_dir',
 'tone_start_timestamp',
 'tone_stop_timestamp',
 'tone_stop_frame',
 'box_1_port_entry_timestamps',
 'tone_start_frame',
 'box_2_port_entry_frames',
 'get_reward_frame',
 'out_reward_frame',
 'first_timestamp',
 'last_timestamp',
 'experiment',
 'all_subjects',
 'notes',
 'condition ',
 'competition_closeness',
 'box_1_port_entry_frames',
 'box_2_port_entry_timestamps',
 'video_name']

In [26]:
dropping_columns = ['tone_start_frame', 'tone_stop_frame', 'condition ', 'notes', 'competition_closeness', '10s_before_tone_frame', '10s_after_tone_frame']

In [27]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

In [28]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [29]:
LFP_SPECTRAL_DF["first_timestamp"].iloc[0]

1830734.0

In [30]:
dropping_columns

['video_name',
 'out_reward_frame',
 'session_dir',
 'first_timestamp',
 'last_timestamp',
 'box_2_port_entry_timestamps',
 'tone_start_timestamp',
 'experiment',
 'box_1_port_entry_frames',
 'get_reward_frame',
 'all_subjects',
 'tone_stop_timestamp',
 'box_2_port_entry_frames',
 'box_1_port_entry_timestamps']

In [31]:
dropping_columns = ['experiment',
                    'box_1_port_entry_frames'
                    'box_1_port_entry_timestamps'
                    'box_2_port_entry_frames'
                    'box_2_port_entry_timestamps',
                    'session_dir',
                    'all_subjects',
                    'first_timestamp',
                    'last_timestamp']


In [32]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=dropping_columns, errors="ignore")

## Merging the LFP and trial dataframe

In [33]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [34]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [35]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [36]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition "])

In [43]:
TRIAL_LABELS_DF["condition "].unique()

array(['rewarded', 3.1, 3.3, 'tie', 4.3, 3.4, 4.4, 5.3, 4.2], dtype=object)

In [37]:
# TRIAL_LABELS_DF = TRIAL_LABELS_DF[~TRIAL_LABELS_DF["condition "].isin(["rewarded"])].copy()

In [38]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [39]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.2'], dtype=object)

In [40]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [44]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,box_2_port_entry_frames,video_name,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
1,0.0,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,NaN,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,NaN,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,NaN,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,5465.0,5545,5565.0,5664.0,rewarded,NaN,NaN,NaN,NaN,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]
5,0.0,6962.0,7042,7062.0,7161.0,rewarded,NaN,NaN,NaN,NaN,[6960 6982] [6996 6996] [6998 7003] [7005 7009...,[6933 6963] [6980 7036] [7053 7053] [7054 7227],20240320_142408_alone_comp_subj_3-1_and_3-3.1,6975965.0,7175963.0,[6974160 6996963] [7009965 7010761] [7012161 7...,[6946160 6977161] [6993361 7050761] [7066559 7...


In [41]:
raise ValueError()

ValueError: 

In [ ]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [ ]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
TRIALS_AND_SPECTRAL_DF.tail()

In [ ]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [ ]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [ ]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

# Calculating the relevant timestamps

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [ ]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

# Filtering out the SLEAP posed for during trials

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

In [ ]:
['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [ ]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [ ]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col]


In [ ]:
sleap_columns

In [ ]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamp"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"]

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_frame_index"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

- Filtering coherence

In [ ]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [ ]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
granger_columns

In [ ]:
for col in sorted(granger_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
power_columns

In [ ]:
for col in sorted(power_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out phase

In [ ]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = phase_columns + band_columns +lfp_columns

In [ ]:
phase_columns

In [ ]:
for col in sorted(phase_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_lfp_timestamps"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape